In [1]:
import pandas as pd
import numpy as np

In [2]:
df_data_villes = pd.read_csv("villes_cp.csv", 
                             delimiter=";",
                             encoding="latin1")

In [4]:
#permet d'avoir 5 caractères dans tous les cas donc 4400 --> 04400
df_data_villes['Code_postal'] = df_data_villes['Code_postal'].astype(str).str.zfill(5)

In [5]:
df_data_villes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39192 entries, 0 to 39191
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   #Code_commune_INSEE     39192 non-null  object
 1   Nom_de_la_commune       39192 non-null  object
 2   Code_postal             39192 non-null  object
 3   Libellé_d_acheminement  39192 non-null  object
 4   Ligne_5                 4804 non-null   object
dtypes: object(5)
memory usage: 1.5+ MB


In [12]:
df_data_villes['departement'] = df_data_villes['Code_postal'].str[:2]

In [8]:
#Création de la colonne pour le scrapping de Meilleur Agent
df_data_villes['Lib_MA'] = df_data_villes['Ligne_5'].fillna(df_data_villes['Nom_de_la_commune'])

In [13]:
# remplace les majuscule par des minuscule et remplacer les espaces par des "-"
df_data_villes['Nom_de_la_commune_url'] = df_data_villes['Lib_MA'].str.lower().str.replace(" ", "-", regex=False) 

# remplacer les st par saint
df_data_villes['Nom_de_la_commune_url'] = df_data_villes['Nom_de_la_commune_url'].str.replace(r'^st', 'saint', case=False, regex=True)

#Création de la colonne url
df_data_villes['url'] = ("https://www.meilleursagents.com/prix-immobilier/" + df_data_villes['Nom_de_la_commune_url'] + "-" + df_data_villes['Code_postal'].astype(str) + "/")

#Je ne garde que les départements de la métropole
df_data_villes_clean = df_data_villes[df_data_villes['Code_postal'] < '96000']

#je supprime les colonnes inutiles
df_data_villes_clean = df_data_villes_clean[['#Code_commune_INSEE','Lib_MA','departement','Code_postal','Nom_de_la_commune_url','url']]

In [20]:
# Liste des départements à filtrer
Pays_de_la_loire = ['44', '49', '53', '85', '72']

# Filtrage
df_data_villes_clean_filtre = df_data_villes_clean[df_data_villes_clean['departement'].isin(Pays_de_la_loire)]

In [21]:
#Test
df_data_villes_clean_filtre

,#Code_commune_INSEE,Lib_MA,departement,Code_postal,Nom_de_la_commune_url,url
17918,44001,ABBARETZ,44,44170,abbaretz,https://www.meilleursagents.com/prix-immobilie...
17919,44002,AIGREFEUILLE SUR MAINE,44,44140,aigrefeuille-sur-maine,https://www.meilleursagents.com/prix-immobilie...
17920,44003,ANCENIS,44,44150,ancenis,https://www.meilleursagents.com/prix-immobilie...
17921,44003,ST GEREON,44,44150,saint-gereon,https://www.meilleursagents.com/prix-immobilie...
17922,44005,ARTHON EN RETZ,44,44320,arthon-en-retz,https://www.meilleursagents.com/prix-immobilie...
...,...,...,...,...,...,...
36551,85302,CHAMBRETAUD,85,85500,chambretaud,https://www.meilleursagents.com/prix-immobilie...
36552,85303,VIX,85,85770,vix,https://www.meilleursagents.com/prix-immobilie...
36553,85304,VOUILLE LES MARAIS,85,85450,vouille-les-marais,https://www.meilleursagents.com/prix-immobilie...
36554,85305,VOUVANT,85,85120,vouvant,https://www.meilleursagents.com/prix-immobilie...


In [22]:
#Ici on enregistre le nouveau df en entier
df_data_villes_clean.to_csv('villes_cp_clean.csv')

In [23]:
#Ici on divise et enregistre le nouveau df en 6
number_of_chunks = 6
for i, chunk in enumerate(np.array_split(df_data_villes_clean_filtre, number_of_chunks)):
    chunk.to_csv(f'pays_de_la_loire{i+1}.csv')

/Users/amauryguilbaud/Library/Python/3.9/lib/python/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
